# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [25]:
import requests
import re
from bs4 import BeautifulSoup
import os

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [26]:
input_keyword = "國立臺灣大學醫學院"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

國立臺灣大學醫學院: %E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E9%86%AB%E5%AD%B8%E9%99%A2
/wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E9%86%AB%E5%AD%B8%E9%99%A2


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [27]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")


#建立filter function，讓content包含內文(p)、標題(h2)、副標題(h3)、項目符號(li)、編號符號中的內容(li)、主條目(div)
#但不包含wiki右方表格與下方外部連結表格
def TextInParagraphNotTable(tag):
    name = ['p', 'h2', 'h3', 'li']
    return ((tag.name in name) or (['hatnote', 'navigation-not-searchable'] in tag.attrs.values())) and not tag.find_parent("table", class_ = ["infobox vcard", "navbox"])

#tag.find("div", class_ = "hatnote navigation-not-searchable")
# or (tag.attrs("class") == "hatnote navigation-not-searchable") 
# and not tag.find_parent("table", class_ = ["infobox vcard", "navbox"]

content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(TextInParagraphNotTable)

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


國立臺灣大學醫學院，簡稱臺大醫學院，為國立臺灣大學學院之一。是台灣最早成立的醫學院校，成立於1897年。

院區位於臺北市中正區中山南路與仁愛路交叉口，建築有日治時期的二號館（今醫學人文博物館）、藥學館及藥理館；二戰後增建的藥理實驗室、聯合教學館、圓形教室以及男二與女六宿舍；最後為整建後的建築物基礎醫學大樓及診斷治療大樓等。

目錄
1 歷史[1]
2 歷任院長
3 組織

3.1 行政業務單位
3.2 附屬機構
3.3 系所
3.4 研究中心


3.1 行政業務單位
3.2 附屬機構
3.3 系所
3.4 研究中心
4 學科
5 參考資料
6 外部連結
歷史[1][編輯]
1895年6月20日，日軍在大稻埕千秋街（今臺北市貴德街）借用民宅建立「大日本臺灣病院」（今國立臺灣大學醫學院附設醫院前身），由濱野昇出任首任院長。
1896年4月，大日本臺灣病院改隸台北縣，因此更名為「臺北病院」。1897年5月，台灣總督府醫院官制公布，台北病院改稱為「臺灣總督府臺北醫院」。
1897年5月，臺灣總督府醫院官制公布，病院再更名為「臺灣總督府臺北醫院」。7月，病院遷移到明石町（今常德街的臨新公園），山口秀高在院內開設「醫學講習所」。
1899年，改建為「臺灣總督府醫學校」。
1918年，改制為「臺灣總督醫學校專門部」。
1919年，成立「臺灣總督府醫學專門學校」。
1922年，專門部與「臺灣總督府醫學專門學校」合併。
1927年，改名「臺灣總督府台北醫學專門學校」。
1936年，成立「臺北帝國大學醫學部」，專門學校併入成為「臺北帝國大學附屬醫學專門部」。
1938年，臺北醫院併入臺北帝大醫學部，改稱「臺北帝國大學醫學部附屬醫院」。
1945年中華民國政府接收臺灣後，更名為「國立臺灣大學醫學院」。 臺北帝國大學醫學部附屬醫院、日本赤十字社臺灣支部病院，分別改為國立臺灣大學醫學院第一附屬醫院、國立臺灣大學醫學院第二附屬醫院。
1946年，奉教育部函，國立台灣大學醫學部改稱醫學院，醫科四年制改為五年制並加一年實習；醫學專門部併入醫學院。此為台灣施行中國式醫學教育制度之始。[2]
1947年，國立臺灣大學醫學院第二附屬醫院（院址為今臺北市立聯合醫院中興院區，院制由今衛生福利部臺北醫院承襲）改隸屬臺灣省行政長官公署，因而移出國立臺灣大學醫學院管理。
1948年，第三十七學年度開學前，教育

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [28]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [國立臺灣大學] /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8
外部連結: [學院] /wiki/%E5%AD%B8%E9%99%A2
外部連結: [台灣] /wiki/%E5%8F%B0%E7%81%A3
外部連結: [醫學院] /wiki/%E9%86%AB%E5%AD%B8%E9%99%A2
外部連結: [院區] /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E9%86%AB%E5%AD%B8%E9%99%A2%E6%A0%A1%E5%8D%80
外部連結: [臺北市] /wiki/%E8%87%BA%E5%8C%97%E5%B8%82
外部連結: [中正區] /wiki/%E4%B8%AD%E6%AD%A3%E5%8D%80_(%E8%87%BA%E5%8C%97%E5%B8%82)
外部連結: [中山南路] /wiki/%E4%B8%AD%E5%B1%B1%E5%8D%97%E8%B7%AF_(%E8%87%BA%E5%8C%97%E5%B8%82)
外部連結: [仁愛路] /wiki/%E4%BB%81%E6%84%9B%E8%B7%AF_(%E5%8F%B0%E5%8C%97%E5%B8%82)
外部連結: [日治] /wiki/%E6%97%A5%E6%B2%BB
外部連結: [醫學人文博物館] /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E9%86%AB%E5%AD%B8%E4%BA%BA%E6%96%87%E5%8D%9A%E7%89%A9%E9%A4%A8
外部連結: [二戰] /wiki/%E4%BA%8C%E6%88%B0
外部連結: [日軍] /wiki/%E6%97%A5%E8%BB%8D
外部連結: [大稻埕] /wiki/%E5%A4%A7%E7%A8%BB%E5%9F%95
外部連結: [國立臺灣大學醫學院附設醫院] /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E9%86%AB%E5%AD%B8%E9%99%A2%E

## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [29]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(TextInParagraphNotTable)
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        
        
        # 決定要儲存的資料夾
        output_dir = './Data/WikiArticle/'
    
        # 假如資料夾不存在就新增一個資料夾
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 解析回傳資料，並萃取文章內容
        with open(output_dir + key_word + ".txt", "w", encoding = "utf-8") as f:
            for paragraph in content:
                
                # 標題前分段
                if paragraph.name == "h2":
                    f.write("\n\n")
                elif paragraph.name == "h3":
                    f.write("\n")
                
                f.write(paragraph.get_text() + "\n")

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        
        # 最後一個遞迴不必儲存下一遞迴的連結資訊，節省效能
        if (recursive == max_recursive_depth):
            return
        else:        
            external_link_dict = dict()

            for ext_link in content:
                a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?![;:#])\S)*$"))
                if len(a_tag) > 0:
                    for link_string in a_tag:
                        a_link = link_string["href"]       # 外部連結的網址
                        a_keyword = link_string.get_text()  # 外部連結的中文名稱
                        #print("外部連結: [%s] %s" % (a_keyword, a_link))

                        # 以{連結的網址為key:外部連結的中文名稱為valueZ}形式儲存
                        external_link_dict[a_link] = a_keyword
                
        #print(external_link_dict)

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [ ]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E9%86%AB%E5%AD%B8%E9%99%A2 (國立臺灣大學醫學院)
遞迴層[1] - /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8 (國立臺灣大學)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E5%A4%A7%E5%B0%88%E9%99%A2%E6%A0%A1%E5%88%97%E8%A1%A8 (臺灣大專院校列表)
遞迴層[2] - /wiki/NTU (NTU)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3 (臺灣)
遞迴層[2] - /wiki/%E7%B6%9C%E5%90%88%E5%A4%A7%E5%AD%B8 (綜合大學)
遞迴層[2] - /wiki/%E9%AB%98%E7%AD%89%E6%95%99%E8%82%B2 (高等教育)
遞迴層[2] - /wiki/%E5%8F%B0%E7%81%A3%E6%97%A5%E6%B2%BB%E6%99%82%E6%9C%9F (日治時期)
遞迴層[2] - /wiki/%E8%87%BA%E5%8C%97%E5%B8%9D%E5%9C%8B%E5%A4%A7%E5%AD%B8 (臺北帝國大學)
遞迴層[2] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B (中華民國)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%88%B0%E5%BE%8C%E6%99%82%E6%9C%9F (收回臺灣後)
遞迴層[2] - /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E9%86%AB%E5%AD%B8%E9%99%A2%E9%99%84%E8%A8%AD%E9%86%AB%E9%99%A2 (臺大醫院)
遞迴層[2] - /wiki/%E9%AB%98%E7%AD%89%E6%95%99%E8%82%B2%E6%B7%B1%E8%80%95%E8%A8%88%E7%95%AB (高教深耕計畫)
遞

遞迴層[2] - /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E8%A1%8C%E6%94%BF%E5%A4%A7%E6%A8%93 (行政大樓)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E6%A0%A1%E9%96%80 (臺灣大學校門)
遞迴層[2] - /wiki/%E7%BD%97%E6%96%AF%E7%A6%8F%E8%B7%AF_(%E5%8F%B0%E5%8C%97%E5%B8%82) (羅斯福路)
遞迴層[2] - /wiki/%E5%82%85%E5%9C%92 (傅園)
遞迴層[2] - /wiki/%E6%BA%AA%E9%A0%AD (溪頭)
遞迴層[2] - /wiki/%E4%BB%81%E6%84%9B%E8%B7%AF_(%E8%87%BA%E5%8C%97%E5%B8%82) (仁愛路)
遞迴層[2] - /wiki/%E8%87%BA%E5%8C%97%E9%86%AB%E5%AD%B8%E5%B0%88%E9%96%80%E5%AD%B8%E6%A0%A1 (臺北醫學專門學校)
遞迴層[2] - /wiki/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8%E7%94%9F%E6%85%8B%E6%B1%A0 (生態池)
遞迴層[2] - /wiki/%E7%91%A0%E5%85%AC%E5%9C%B3 (瑠公圳)
遞迴層[2] - /wiki/%E5%9F%BA%E9%9A%86%E8%B7%AF (基隆路)
遞迴層[2] - /wiki/%E8%87%BA%E5%8C%97%E5%B8%82%E6%94%BF%E5%BA%9C (臺北市政府)
遞迴層[2] - /wiki/%E5%85%AC%E8%B7%AF (公共道路)
遞迴層[2] - /wiki/%E9%B9%BF%E9%B3%B4%E5%A0%82 (鹿鳴堂)
遞迴層[2] - /wiki/%E5%85%B1%E5%90%8C%E4%B8%89%E6%9D%BE (共同三松)
遞迴層[2] - /wiki/%E5%85%B1%E5%90%8C%E6%95%99%E

遞迴層[2] - /wiki/%E4%BD%99%E5%85%89%E4%B8%AD (余光中)
遞迴層[2] - /wiki/%E8%97%8D%E6%98%9F%E8%A9%A9%E7%A4%BE (藍星詩社)
遞迴層[2] - /wiki/%E6%9E%97%E6%96%87%E6%9C%88 (林文月)
遞迴層[2] - /wiki/%E5%AE%8B%E7%91%9E%E6%A8%93 (宋瑞樓)
遞迴層[2] - /wiki/%E6%9E%97%E9%95%B7%E5%A3%BD (林長壽)
遞迴層[2] - /wiki/%E6%9C%B1%E6%95%AC%E4%B8%80 (朱敬一)
遞迴層[2] - /wiki/%E6%9C%B1%E5%9C%8B%E7%91%9E (朱國瑞)
遞迴層[2] - /wiki/%E9%AE%91%E4%BA%A6%E8%88%88 (鮑亦興)
遞迴層[2] - /wiki/%E6%9D%8E%E7%BE%85%E6%AC%8A (李羅權)
遞迴層[2] - /wiki/%E9%99%B3%E5%AE%9A%E4%BF%A1 (陳定信)
遞迴層[2] - /wiki/%E5%BB%96%E4%B8%80%E4%B9%85 (廖一久)
遞迴層[2] - /wiki/%E6%9D%8E%E5%A3%AC%E7%99%B8 (李壬癸)
遞迴層[2] - /wiki/%E9%8C%A2%E7%85%A6 (錢煦)
遞迴層[2] - /wiki/%E5%90%B3%E6%88%90%E6%96%87 (吳成文)
遞迴層[2] - /wiki/%E5%BD%AD%E6%97%AD%E6%98%8E (彭旭明)
遞迴層[2] - /wiki/%E7%9B%A7%E5%BF%97%E9%81%A0 (盧志遠)
遞迴層[2] - /wiki/%E8%B3%B4%E6%98%8E%E8%A9%94 (賴明詔)
遞迴層[2] - /wiki/%E9%82%B1%E8%81%AF%E6%81%AD (邱聯恭)
遞迴層[2] - /wiki/%E8%83%A1%E4%BD%9B_(%E5%8F%B0%E7%81%A3) (胡佛)
遞迴層[2] - /wiki/%E9%BB%83%E8%8C%82%E6%A6%AE (黃茂榮)
遞迴層[2] - 

遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E8%AA%9E%E8%A8%80%E5%88%97%E8%A1%A8 (臺灣語言列表)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%96%87%E5%AD%B8 (臺灣文學)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E5%AE%97%E6%95%99 (臺灣宗教)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E5%96%AA%E8%91%AC (臺灣喪葬)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E5%8F%A4%E8%B9%9F (臺灣古蹟)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E9%81%BA%E5%9D%80 (臺灣遺址)
遞迴層[2] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E5%9C%8B%E5%AF%B6 (中華民國國寶)
遞迴層[2] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E6%B0%91%E4%BF%97%E5%8F%8A%E6%9C%89%E9%97%9C%E6%96%87%E7%89%A9 (民俗文物)
遞迴層[2] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%AF%80%E6%97%A5%E8%88%87%E6%AD%B2%E6%99%82%E5%88%97%E8%A1%A8 (中華民國節日與歲時列表)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%95%99%E8%82%B2 (臺灣教育)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E9%AB%94%E8%82%B2 (臺灣體育)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E5%AA%92%E9%AB%94 (臺灣媒體)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E9%83%B5%E7%A5%A8 (臺灣郵票)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E9%A3%B2%E9%A3%9F (臺灣飲食)
遞

遞迴層[2] - /wiki/%E9%80%9A%E8%A8%8A (通訊)
遞迴層[2] - /wiki/%E7%94%B5%E5%AD%90%E5%B7%A5%E4%B8%9A (電子)
遞迴層[2] - /wiki/%E9%AB%98%E7%A7%91%E6%8A%80 (高科技)
遞迴層[2] - /wiki/%E5%A4%96%E6%B1%87 (外匯)
遞迴層[2] - /wiki/%E6%9C%8D%E5%8B%99%E6%A5%AD (服務業)
遞迴層[2] - /wiki/%E6%96%87%E5%8C%96%E4%BA%A7%E4%B8%9A (文化產業)
遞迴層[2] - /wiki/%E6%97%85%E6%B8%B8 (觀光業)
遞迴層[2] - /wiki/%E6%88%92%E5%9A%B4%E6%99%82%E4%BB%A3 (戒嚴時代)
遞迴層[2] - /wiki/%E8%87%AA%E7%94%B1%E5%8C%96 (自由化)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%B0%91%E4%B8%BB%E5%8C%96 (民主化)
遞迴層[2] - /wiki/%E4%B8%AD%E5%9C%8B%E5%9C%8B%E6%B0%91%E9%BB%A8 (中國國民黨)
遞迴層[2] - /wiki/%E6%B0%91%E4%B8%BB%E9%80%B2%E6%AD%A5%E9%BB%A8 (民主進步黨)
遞迴層[2] - /wiki/%E6%94%BF%E9%BB%A8%E6%94%BF%E6%B2%BB (政黨政治)
遞迴層[2] - /wiki/%E4%B8%AD%E5%9C%8B%E7%B5%B1%E4%B8%80 (中國統一)
遞迴層[2] - /wiki/%E5%8F%B0%E7%81%A3%E7%8D%A8%E7%AB%8B%E9%81%8B%E5%8B%95 (獨)
遞迴層[2] - /wiki/%E5%85%AC%E6%B0%91%E7%A4%BE%E6%9C%83 (公民社會)
遞迴層[2] - /wiki/%E4%B8%AD%E5%9C%8B%E5%8F%B2%E8%A7%80 (中國史觀)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E4%B8%BB%E9

遞迴層[2] - /wiki/%E6%97%A5%E6%B2%BB%E8%87%BA%E7%81%A3%E6%AD%B7%E5%8F%B2%E5%B9%B4%E8%A1%A8 (日治臺灣歷史年表)
遞迴層[2] - /wiki/%E6%88%B0%E5%BE%8C%E8%87%BA%E7%81%A3%E6%AD%B7%E5%8F%B2%E5%B9%B4%E8%A1%A8 (戰後臺灣歷史年表)
遞迴層[2] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%B8%BD%E7%B5%B1%E7%9B%B4%E9%81%B8%E5%88%97%E8%A1%A8 (中華民國總統直選列表)
遞迴層[2] - /wiki/%E5%8F%B0%E6%B9%BE%E5%8F%B2 (臺灣史)
遞迴層[2] - /wiki/%E6%9D%B1%E4%BA%9E%E5%8F%B2 (東亞史)
遞迴層[2] - /wiki/%E4%B8%96%E7%95%8C%E5%8F%B2 (世界史)
遞迴層[2] - /wiki/%E6%96%B0%E5%B8%9D%E5%9C%8B%E4%B8%BB%E7%BE%A9 (新帝國主義)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%8A%97%E6%97%A5%E9%81%8B%E5%8B%95 (臺灣抗日運動)
遞迴層[2] - /wiki/%E5%A4%AA%E5%B9%B3%E6%B4%8B%E6%88%B0%E7%88%AD (太平洋戰爭)
遞迴層[2] - /wiki/%E7%AC%AC%E4%BA%8C%E6%AC%A1%E5%9C%8B%E5%85%B1%E5%85%A7%E6%88%B0 (第二次國共內戰)
遞迴層[2] - /wiki/%E4%BA%8C%E4%BA%8C%E5%85%AB%E4%BA%8B%E4%BB%B6 (二二八事件)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E7%99%BD%E8%89%B2%E6%81%90%E6%80%96%E6%99%82%E6%9C%9F (白色恐怖)
遞迴層[2] - /wiki/%E7%BE%8E%E9%BA%97%E5%B3%B6%E4%BA%8B%E4%BB%B6 (美麗島事

遞迴層[2] - /wiki/%E8%90%AC%E5%B9%B4%E7%B8%A3_(%E8%87%BA%E7%81%A3) (萬年縣))
遞迴層[2] - /wiki/%E5%BB%88%E9%96%80 (廈門)
遞迴層[2] - /wiki/%E9%84%AD%E8%A5%B2_(%E6%98%8E%E9%84%AD) (鄭襲)
遞迴層[2] - /wiki/%E4%B8%8D%E5%88%97%E9%A1%9B%E6%9D%B1%E5%8D%B0%E5%BA%A6%E5%85%AC%E5%8F%B8 (英國東印度公司)
遞迴層[2] - /wiki/%E8%8B%B1%E5%9B%BD (英國)
遞迴層[2] - /wiki/%E8%8B%B1%E5%9C%8B%E4%BA%BA (英國人)
遞迴層[2] - /wiki/%E9%99%B3%E6%B0%B8%E8%8F%AF_(%E6%98%8E%E6%9C%9D%E5%B0%87%E9%A0%98) (陳永華)
遞迴層[2] - /wiki/%E4%B8%AD%E5%A4%AE%E6%94%BF%E5%BA%9C (中央政府)
遞迴層[2] - /wiki/%E5%AD%94%E5%BA%99 (孔廟)
遞迴層[2] - /wiki/%E7%A7%91%E4%B8%BE (科舉)
遞迴層[2] - /wiki/%E9%84%AD%E5%85%8B%E8%87%A7 (鄭克臧)
遞迴層[2] - /wiki/%E9%A6%AE%E9%8C%AB%E7%AF%84 (馮錫範)
遞迴層[2] - /wiki/%E5%8A%89%E5%9C%8B%E8%BB%92 (劉國軒)
遞迴層[2] - /wiki/%E6%94%BF%E8%AE%8A (政變)
遞迴層[2] - /wiki/%E9%84%AD%E5%85%8B%E5%A1%BD (鄭克塽)
遞迴層[2] - /wiki/%E6%BE%8E%E6%B9%96%E6%B5%B7%E6%88%98 (澎湖海戰)
遞迴層[2] - /wiki/%E6%96%BD%E7%90%85 (施琅)
遞迴層[2] - /wiki/%E5%A7%9A%E5%95%9F%E8%81%96 (姚啟聖)
遞迴層[2] - /wiki/%E5%BA%B7%E7%86%99%E5%

遞迴層[2] - /wiki/%E7%9F%A5%E8%AD%98%E5%88%86%E5%AD%90 (知識分子)
遞迴層[2] - /wiki/%E5%8B%9E%E5%B7%A5%E9%81%8B%E5%8B%95 (勞工運動)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%96%87%E5%8C%96%E5%8D%94%E6%9C%83 (臺灣文化協會)
遞迴層[2] - /wiki/%E8%87%BA%E7%81%A3%E6%B0%91%E7%9C%BE%E9%BB%A8_(1927%E5%B9%B4) (臺灣民眾黨)
遞迴層[2] - /wiki/%E5%B7%A6%E6%B4%BE (左)
